In [1]:
from utils.account_status import AccountStatus

account_status = AccountStatus('accounts.csv')

row = account_status.get_account_info_by_plate('Y0V029')

print(row)

plate='Y0V029' ch_code='01RBA2404-4' endo_date='2024-04-03' client='ROB AUTO' car_model='2022 MITSUBISHI XPANDER CROSS AT' plate_number_normalized='Y0V029'


In [3]:
from utils.is_similar_plate import is_similar_plate

plate1 = "ABC1234"
plate2 = "ABC123"

is_similar_plate(plate1, plate2)

True

In [ ]:
from lark.token_manager import TokenManager
import os
from dotenv import load_dotenv

load_dotenv()

chopper_app_id = os.getenv('CHOPPER_APP_ID')
chopper_app_secret = os.getenv('CHOPPER_APP_SECRET')

token_manager = TokenManager(app_id=chopper_app_id, app_secret=chopper_app_secret)

tenant_token = token_manager.get_tenant_access_token_sync()

In [ ]:
from utils.get_group_members import get_group_members

members = get_group_members(tenant_token.tenant_access_token)

In [2]:
import pandas as pd
from database.database import engine


df = pd.read_sql(
    """
        SELECT count(name) as total_requests, name, current_date FROM logs
        WHERE current_date = DATE('now')
        GROUP BY name
    """,
    con=engine
)

print(df)


   total_requests               name current_date
0               3  CALIXTO MALLORCA    2024-12-05
1               6             Jerome   2024-12-05
2              32      Julius Tuliao   2024-12-05
3               4  calixto Mallorca    2024-12-05
4              16             jerome   2024-12-05
5               1               null   2024-12-05


{
    "code": 0,
    "data": {
        "body": {
            "content": "{\"text\":\"test notification for buzz. kindly disregard tenk u\"}"
        },
        "chat_id": "oc_b61235e79bac3d17d9107433e8267154",
        "create_time": "1733123840768",
        "deleted": false,
        "message_id": "om_a5aa991b5636a59ad99782d13dafb06f",
        "msg_type": "text",
        "sender": {
            "id": "cli_a6889e23ee38500a",
            "id_type": "app_id",
            "sender_type": "app",
            "tenant_key": "12f9cd33134f1759"
        },
        "update_time": "1733123840768",
        "updated": false
    },
    "msg": "success"
}


In [ ]:
from typing import List

print('chat_id', message.data.message_id)

def buzz_this_message(
    tenant_token: str,
    message_id: str,
    group_members_union_id: List[str]
):
    formatted_url = "https://open.larksuite.com/open-apis/im/v1/messages/{message_id}/urgent_app".format(message_id=message_id)

    headers = {
        "Authorization": f"Bearer {tenant_token}"
    }

    params = {
        "user_id_type": "union_id"
    }

    body = {
        "user_id_list": group_members_union_id
    }
    
    response = requests.patch(
        url=formatted_url, 
        json=body, 
        headers=headers, 
        params=params
    )


members = get_group_members(
    tenant_access_token=tenant_token.tenant_access_token,
)

members = [member.member_id for member in members.data.items]

buzz_this_message(
    tenant_token=tenant_token.tenant_access_token,
    message_id=message.data.message_id,
    group_members_union_id=members
)

chat_id om_0bb2f940bc3383fde466424fc7d1dd31
https://open.larksuite.com/open-apis/im/v1/messages/om_0bb2f940bc3383fde466424fc7d1dd31/urgent_app
b'{"code":0,"data":{"invalid_user_id_list":[]},"msg":"success"}'


In [31]:
import requests
import os
from typing import Literal, Optional
from pydantic import BaseModel


class UploadImageMessageDataField(BaseModel):
    image_key: str


class UploadImageMessageResponse(BaseModel):
    code: int
    msg: str
    data: Optional[UploadImageMessageDataField] = None


MAX_SIZE_MB = 10

def upload_image_gc(
    tenant_token: str,
    image_path: str,
    image_type: Literal['avatar', 'message'] = 'message'
):
    url = "https://open.larksuite.com/open-apis/im/v1/images"

    # Check if file exists
    if not os.path.isfile(image_path):
        raise FileNotFoundError(f"The file '{image_path}' does not exist.")

    # Check file size
    file_size = os.path.getsize(image_path)
    if file_size > MAX_SIZE_MB * 1024 * 1024:
        raise ValueError(f"Image size exceeds {MAX_SIZE_MB}MB limit.")

     # Prepare headers
    headers = {
        "Authorization": f"Bearer {tenant_token}",
    }

    # Prepare multipart/form-data
    with open(image_path, 'rb') as image_file:
        files = {
            "image": (os.path.basename(image_path), image_file, "application/octet-stream"),
        }

        data = {
            "image_type": image_type
        }

        try:
            response = requests.post(
                url, 
                headers=headers, 
                files=files, 
                data=data
            )

            return UploadImageMessageResponse(**response.json())
        except requests.exceptions.RequestException as e:
            raise Exception(f"HTTP request failed: {e}")

NameError: name 'upload_image_gc' is not defined

In [18]:
from datetime import datetime, date

# Get today's date
today = date.today()

# Convert today's date to a datetime object (with time set to 00:00:00)
today_datetime = datetime.combine(today, datetime.min.time())

# Get the integer timestamp (seconds since Unix epoch)
timestamp = int(today_datetime.timestamp()) * 1000

print(timestamp)


1733673600000


In [ ]:

# )